reference:https://keras.io/examples/nlp/lstm_seq2seq/

https://arxiv.org/pdf/1409.0473.pdf

reference: https://machinelearningmastery.com/define-encoder-decoder-sequence-sequence-model-neural-machine-translation-keras/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import os

In [ ]:
!pip install wandb
import wandb
from wandb.keras import WandbCallback

In [ ]:
def load_data(path):
    with open(path) as fil:
        data = pd.read_csv(fil,sep='\t',header=None,names=["hi","en",""],skip_blank_lines=True,index_col=None)
    data = data[['hi','en']]
    return data

In [ ]:
# Download the dataset
!curl https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar --output daksh.tar
# Extract the downloaded tar file
!tar -xvf  'daksh.tar'
# Set the file paths to train, validation and test dataset
train_file=os.path.join(os.getcwd(),"dakshina_dataset_v1.0","hi","lexicons","hi.translit.sampled.train.tsv")
vaildation_file_path = os.path.join(os.getcwd(),"dakshina_dataset_v1.0","hi","lexicons","hi.translit.sampled.dev.tsv")
test_file_path = os.path.join(os.getcwd(),"dakshina_dataset_v1.0","hi","lexicons","hi.translit.sampled.test.tsv")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1915M  100 1915M    0     0   227M      0  0:00:08  0:00:08 --:--:--  252M
dakshina_dataset_v1.0/bn/
dakshina_dataset_v1.0/bn/lexicons/
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.test.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.train.tsv
dakshina_dataset_v1.0/bn/lexicons/bn.translit.sampled.dev.tsv
dakshina_dataset_v1.0/bn/native_script_wikipedia/
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.valid.text.shuf.txt.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-full.info.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.info.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.text.sorted.tsv.gz
dakshina_dataset_v1.0/bn/native_script_wikipedia/bn.wiki-filt.train.text.shuf.txt.gz
dakshina_dataset_v1.0/bn/native_script

In [ ]:
train_1 = load_data("/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.train.tsv")
dev_1 = load_data("/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.dev.tsv")
test_1 = load_data("/content/dakshina_dataset_v1.0/hi/lexicons/hi.translit.sampled.test.tsv")

In [ ]:
train_1 = train_1[train_1['hi'].notna()]
train_1= train_1[train_1['en'].notna()]
dev_1 = dev_1[dev_1['hi'].notna()]
dev_1= dev_1[dev_1['en'].notna()]
test_1 = test_1[test_1['hi'].notna()]
test_1= test_1[test_1['en'].notna()]

In [ ]:
train_1.head()

,hi,en
0,अं,an
1,अंकगणित,ankganit
2,अंकल,uncle
3,अंकुर,ankur
4,अंकुरण,ankuran


In [ ]:
y = train_1['hi'].values
x = train_1['en'].values
y = '\t'+y+'\n'

In [ ]:
x

array(['an', 'ankganit', 'uncle', ..., 'hyensang', 'xuanzang', 'om'],
      dtype=object)

In [ ]:
y

array(['\tअं\n', '\tअंकगणित\n', '\tअंकल\n', ..., '\tह्वेनसांग\n',
       '\tह्वेनसांग\n', '\tॐ\n'], dtype=object)

In [ ]:
xxl=[xx for xx,yy in zip(x,y)]
english_char = set()
hindi_char = set()
for i in range(0,len(xxl)):
  w1=xxl[i]
  #print(w1)
  for l in str(w1):
    english_char.add(l)
print(english_char)             #the set of all the letters

yyl=[yy for xx,yy in zip(x,y)]

for i in range(0,len(yyl)):
  w2=yyl[i]
  #print(w1)
  for l in str(w2):
    hindi_char.add(l)
print(hindi_char)    

{'i', 'r', 'y', 'o', 'v', 'p', 's', 'n', 'b', 'k', 't', 'h', 'j', 'a', 'q', 'd', 'z', 'e', 'f', 'l', 'g', 'x', 'w', 'm', 'c', 'u'}
{'अ', 'प', 'ौ', 'ठ', 'ण', 'य', 'ल', '\t', 'ु', '्', 'ऊ', 'थ', 'ओ', 'ॉ', 'झ', 'ऋ', 'त', 'छ', 'र', 'े', 'ू', 'उ', 'ॐ', 'ब', 'ए', '\n', 'ि', 'इ', 'ृ', 'ध', 'म', 'श', 'ं', 'ड', 'ख', 'क', 'ै', 'ह', 'ः', 'द', 'ज', 'ा', 'न', 'ग', 'ढ', 'ँ', 'ी', 'ट', 'ई', 'व', 'ञ', 'आ', '़', 'च', 'ङ', 'औ', 'ष', 'ॅ', 'भ', 'घ', 'फ', 'ऐ', 'ऑ', 'स', 'ो'}


In [ ]:
english_char

{'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z'}

In [ ]:
hindi_char

{'\t',
 '\n',
 'ँ',
 'ं',
 'ः',
 'अ',
 'आ',
 'इ',
 'ई',
 'उ',
 'ऊ',
 'ऋ',
 'ए',
 'ऐ',
 'ऑ',
 'ओ',
 'औ',
 'क',
 'ख',
 'ग',
 'घ',
 'ङ',
 'च',
 'छ',
 'ज',
 'झ',
 'ञ',
 'ट',
 'ठ',
 'ड',
 'ढ',
 'ण',
 'त',
 'थ',
 'द',
 'ध',
 'न',
 'प',
 'फ',
 'ब',
 'भ',
 'म',
 'य',
 'र',
 'ल',
 'व',
 'श',
 'ष',
 'स',
 'ह',
 '़',
 'ा',
 'ि',
 'ी',
 'ु',
 'ू',
 'ृ',
 'ॅ',
 'े',
 'ै',
 'ॉ',
 'ो',
 'ौ',
 '्',
 'ॐ'}

In [ ]:
dct_eng= [(ch,i+1) for i,ch in enumerate(english_char)]
english_char_map = dict(dct_eng)

dct_hin= [(ch,i+1) for i,ch in enumerate(hindi_char)]
hindi_char_map = dict(dct_hin)

In [ ]:
english_char_map

{'a': 14,
 'b': 9,
 'c': 25,
 'd': 16,
 'e': 18,
 'f': 19,
 'g': 21,
 'h': 12,
 'i': 1,
 'j': 13,
 'k': 10,
 'l': 20,
 'm': 24,
 'n': 8,
 'o': 4,
 'p': 6,
 'q': 15,
 'r': 2,
 's': 7,
 't': 11,
 'u': 26,
 'v': 5,
 'w': 23,
 'x': 22,
 'y': 3,
 'z': 17}

In [ ]:
num=0
hindi_char_map[" "] = num
english_char_map[" "] = num

In [ ]:
english_char_map

{' ': 0,
 'a': 14,
 'b': 9,
 'c': 25,
 'd': 16,
 'e': 18,
 'f': 19,
 'g': 21,
 'h': 12,
 'i': 1,
 'j': 13,
 'k': 10,
 'l': 20,
 'm': 24,
 'n': 8,
 'o': 4,
 'p': 6,
 'q': 15,
 'r': 2,
 's': 7,
 't': 11,
 'u': 26,
 'v': 5,
 'w': 23,
 'x': 22,
 'y': 3,
 'z': 17}

In [ ]:
len_m_eng= [len(i) for i in x]
max_eng_len = max(len_m_eng)
len_m_hin= [len(i) for i in y]
max_hin_len = max(len_m_hin)

In [ ]:
max_eng_len

20

In [ ]:
def process(data):
    x,y = data['en'].values, data['hi'].values
    y = "\t" + y + "\n"
    
    a = np.zeros((len(x),max_eng_len),dtype="float32")
    b = np.zeros((len(y),max_hin_len),dtype="float32")
    c = np.zeros((len(y),max_hin_len,len(hindi_char)+1),dtype="int")
    
    
    for i,(xx,yy) in enumerate(zip(x,y)):
        for j,ch in enumerate(xx):
            a[i,j] = english_char_map[ch]

        a[i,j+1:] = english_char_map[" "]
        for j,ch in enumerate(yy):
            b[i,j] = hindi_char_map[ch]

            if j>0:
                c[i,j-1,hindi_char_map[ch]] = 1

        b[i,j+1:] = hindi_char_map[" "]
        c[i,j:,hindi_char_map[" "]] = 1
        
    return a,b,c

In [ ]:
trainx, trainxx, trainy = process(train_1)
valx, valxx, valy = process(dev_1)
testx,testxx,testy = process(test_1)

In [ ]:
testx

array([[14.,  8., 10., ...,  0.,  0.,  0.],
       [14.,  8., 10., ...,  0.,  0.,  0.],
       [14.,  8., 10., ...,  0.,  0.,  0.],
       ...,
       [12.,  4.,  7., ...,  0.,  0.,  0.],
       [12.,  4.,  7., ...,  0.,  0.,  0.],
       [12.,  4.,  7., ...,  0.,  0.,  0.]], dtype=float32)

In [ ]:
np.random.seed(25)

In [ ]:
rev_dst_eng= [(i,char) for char,i in english_char_map.items()]
reverse_eng_map = dict(rev_dst_eng)
rev_dst_hin= [(i,char) for char,i in hindi_char_map.items()]
reverse_hin_map = dict(rev_dst_hin)

In [ ]:
reverse_eng_map

{0: ' ',
 1: 'i',
 2: 'r',
 3: 'y',
 4: 'o',
 5: 'v',
 6: 'p',
 7: 's',
 8: 'n',
 9: 'b',
 10: 'k',
 11: 't',
 12: 'h',
 13: 'j',
 14: 'a',
 15: 'q',
 16: 'd',
 17: 'z',
 18: 'e',
 19: 'f',
 20: 'l',
 21: 'g',
 22: 'x',
 23: 'w',
 24: 'm',
 25: 'c',
 26: 'u'}

In [ ]:
import keras
import tensorflow as tf

In [ ]:
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
from keras.layers import SimpleRNN,LSTM,GRU,Embedding,Dense,Dropout,Input,Concatenate
from tensorflow.keras.optimizers import Adam,Nadam
from keras import Model

In [ ]:
class Attention(tf.keras.layers.Layer):
    def __init__(self, units,name=None, **kwargs):
        super(Attention, self).__init__(name=name)
        self.W1 = Dense(units,name='w1')
        self.W2 = Dense(units,name='w2')
        self.V = Dense(1,name='v')
        self.units = units
        super(Attention, self).__init__(**kwargs)

    def call(self, tup):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query = tup[0]
        values = tup[1]
        query_with_time_axis = tf.expand_dims(query, 1)
        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        

        return context_vector, attention_weights
    
        
    def get_config(self):
        config = super(Attention,self).get_config()
        config.update({
            'units': self.units
        })
        return config

In [ ]:
def lstm_model(cell = "LSTM",nunits = 32, enc_layers = 1, dec_layers = 1,embed_dim = 32,dense_size=32,dropout=None):
    keras.backend.clear_session()
    encoder_inputs = Input(shape=(None,),name='input1')
    encoder_embedding = Embedding(input_dim=len(english_char)+1,output_dim = embed_dim,mask_zero=True,name="enc_embed")
    encoder_context = encoder_embedding(encoder_inputs)
    decoder_inputs = Input(shape=(None,),name='input2')
    decoder_embedding = Embedding(input_dim = len(hindi_char)+1,output_dim = embed_dim,mask_zero=True,name="dec_embed")
    decoder_context = decoder_embedding(decoder_inputs)
    attention = Attention(nunits,name='attention')
    tot_out = []
    concat1 = Concatenate(axis=-1,name='concat1')
    concat2 = Concatenate(axis=1,name='concat2')    
    if cell == "LSTM":
        encoder_prev = [LSTM(nunits,return_sequences=True,name=f"enc_{i}") for i in range(enc_layers-1)]
        encoder_fin = LSTM(nunits,return_sequences=True,return_state=True,name=f"enc_{enc_layers-1}")
        temp = encoder_context
        for x,lay in enumerate(encoder_prev):
            temp = lay(temp)
            if dropout is not None:
                temp = Dropout(dropout,name=f'do_{x}')(temp)
            
        enc_out = encoder_fin(temp)
        dec_states = enc_out[1:]
        
        decoder = [LSTM(nunits,return_sequences=True,return_state=True,name=f"dec_{i}") for i in range(dec_layers)]
        temp_states = [dec_states]*dec_layers
        
        for i in range(max_hin_len):
            tup = (temp_states[0][0],enc_out[0])
            context,att_wts = attention(tup)
            temp = concat1([tf.expand_dims(context, 1), decoder_context[:,i:i+1,:]])
#             temp = tf.expand_dims(temp, 1)
            for i in range(dec_layers):
                temp,sh,sc = decoder[i](temp,initial_state=temp_states[i])
                temp_states[i] = [sh,sc]
            tot_out.append(temp)
            
        outt = concat2(tot_out)
       

            
        
    dense_lay1 = Dense(dense_size,activation='relu',name='dense1')
    pre_out = dense_lay1(outt)
    dense_lay2 = Dense(len(hindi_char)+1,activation = 'softmax',name='dense2')
    final_output = dense_lay2(pre_out)
    
    train = Model([encoder_inputs,decoder_inputs],final_output)
    
    return train
#     return train

In [ ]:
def GRU_model(cell = "GRU",nunits = 32, enc_layers = 1, dec_layers = 1,embed_dim = 32,dense_size=32,dropout=None):
    keras.backend.clear_session()
    encoder_inputs = Input(shape=(None,),name='input1')
    encoder_embedding = Embedding(input_dim=len(english_char)+1,output_dim = embed_dim,mask_zero=True,name="enc_embed")
    encoder_context = encoder_embedding(encoder_inputs)
    decoder_inputs = Input(shape=(None,),name='input2')
    decoder_embedding = Embedding(input_dim = len(hindi_char)+1,output_dim = embed_dim,mask_zero=True,name="dec_embed")
    decoder_context = decoder_embedding(decoder_inputs)
    attention = Attention(nunits,name='attention')
    tot_out = []
    concat1 = Concatenate(axis=-1,name='concat1')
    concat2 = Concatenate(axis=1,name='concat2')    
    if cell == "GRU":
        encoder_prev = [GRU(nunits,return_sequences=True,name=f"enc_{i}") for i in range(enc_layers-1)]
        encoder_fin = GRU(nunits,return_sequences=True,return_state=True,name=f"enc_{enc_layers-1}")
        temp = encoder_context
        for x,lay in enumerate(encoder_prev):
            temp = lay(temp)
            if dropout is not None:
                temp = Dropout(dropout,name=f'do_{x}')(temp)
            
        enc_out = encoder_fin(temp)
        dec_states = enc_out[1:]
        
        decoder = [GRU(nunits,return_sequences=True,return_state=True,name=f"dec_{i}") for i in range(dec_layers)]
        temp_states = []
        for _ in range(dec_layers):
            temp_states += dec_states
        
        for i in range(max_hin_len):
            tup = (temp_states[0],enc_out[0])
            context,att_wts = attention(tup)
            temp = concat1([tf.expand_dims(context, 1), decoder_context[:,i:i+1,:]])
#             temp = tf.expand_dims(temp, 1)
            for i in range(dec_layers):
                temp,st = decoder[i](temp,initial_state=temp_states[i])
                temp_states[i] = st
            tot_out.append(temp)
            
        outt = concat2(tot_out)
       

            
        
    dense_lay1 = Dense(dense_size,activation='relu',name='dense1')
    pre_out = dense_lay1(outt)
    dense_lay2 = Dense(len(hindi_char)+1,activation = 'softmax',name='dense2')
    final_output = dense_lay2(pre_out)
    
    train = Model([encoder_inputs,decoder_inputs],final_output)
    
    return train

In [ ]:
#for wandb
sweep_config = {
    'method': 'bayes', #grid, random, bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'cell': {
            'values': ["LSTM","GRU"]
        },
        'embed_size': {
            'values': [32,128,256]
        },
        'lr': {
            'values': [1e-2,1e-3]
        },
        'dense_size': {
            'values': [64,128,512]
        },
        'dropout': {
            'values': [0.0,0.2,0.5]
        },
        'nunits': {
            'values': [128,256]
        },
        'enc_layers': {
            'values': [1,3,5]
        },
        'dec_layers': {
            'values': [1,3,5]
        },
        
    }
}


In [ ]:
#sweep_id = wandb.sweep(sweep_config,project="a=3", entity="dl22")

In [ ]:
def train():
    # Default values for hyper-parameters we're going to sweep over
    config_defaults = {
        'lr': 1e-2,
        'dense_size': 64,
        'nunits': 128,
        'cell': 'LSTM',
        'embed_size': 64,
        'enc_layers': 1,
        'dec_layers': 1,
        'dropout': 0.
    }

    # Initialize a new wandb run
    wandb.init(config=config_defaults,name="cs6910")
    
    cfg = wandb.config
    
    name = f'nunits_{cfg.nunits}_cell_{cfg.cell}_encl_{cfg.enc_layers}_decl_{cfg.dec_layers}_emb_{cfg.embed_size}_ds_{cfg.dense_size}_do_{cfg.dropout}_lr_{cfg.lr}'
    wandb.run.name = name
    wandb.run.save()
    
    # Config is a variable that holds and saves hyperparameters and inputs
    
    train,enc,dec = lstm_model(nunits=cfg.nunits,
                                dense_size=cfg.dense_size,
                                enc_layers=cfg.enc_layers,
                                dec_layers=cfg.dec_layers,
                                cell = cfg.cell,
                                dropout = cfg.dropout,
                                embed_dim = cfg.embed_size)
    train.compile(optimizer = Adam(lr=cfg.lr),loss='categorical_crossentropy',metrics=['accuracy'])
    print("model building done")
    train.fit([trainx,trainxx],trainy,
             batch_size=64,
             validation_data = ([valx,valxx],valy),
             epochs=10,
             callbacks = [WandbCallback(monitor='val_accuracy',mode='min')])
    print("model training done")


In [ ]:
#wandb.agent(sweep_id, train,count=25)

In [ ]:
def accuracy1(real,pred):
    real = tf.math.argmax(real,axis=2)
    pred = tf.math.argmax(pred,axis=2)
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    acc = tf.math.equal(real,pred)
    mask = tf.cast(mask, dtype='int32')
    acc = tf.cast(acc, dtype='int32')
    acc = tf.math.multiply(acc,mask)
    mask = tf.reduce_sum(mask,axis=1)
    acc = tf.reduce_sum(acc,axis=1)
    acc = tf.math.equal(acc,mask)
    acc = tf.cast(acc, dtype='float32')
    return tf.reduce_mean(acc)

In [ ]:
train = lstm_model(nunits=256,
                                dense_size=512,
                                enc_layers=3,
                                dec_layers=1,
                                cell = "LSTM",
                                dropout = 0.2,
                                embed_dim = 256)
train.compile(optimizer = Adam(lr=1e-3),loss='categorical_crossentropy',metrics=[accuracy1])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
model_cb = tf.keras.callbacks.ModelCheckpoint('best_model.h5',monitor='val_accuracy1',mode='max',save_best_only=True,verbose=1)

In [ ]:
train.fit([trainx,trainxx],trainy,
             batch_size=64,
             validation_data = ([valx,valxx],valy),
             epochs=1,
             callbacks = [model_cb])

691/691 [==============================] - ETA: 0s - loss: 0.8907 - accuracy1: 0.0016
Epoch 1: val_accuracy1 improved from -inf to 0.01404, saving model to best_model.h5
691/691 [==============================] - 674s 889ms/step - loss: 0.8907 - accuracy1: 0.0016 - val_loss: 0.5166 - val_accuracy1: 0.0140


In [ ]:
train.save('best_model_attention.h5')

In [ ]:
def GRU_inference(model,nunits=32,enc_layers=1,dec_layers=1,cell='LSTM',dropout=None):
    keras.backend.clear_session()
    encoder_inputs = model.input[0]
    encoder_embedding = model.get_layer('enc_embed')
    encoder_context = encoder_embedding(encoder_inputs)
    decoder_inputs = model.input[1]
    decoder_embedding = model.get_layer('dec_embed')
    decoder_context = decoder_embedding(decoder_inputs)
    
    
    encoder_prev = [model.get_layer(f'enc_{i}') for i in range(enc_layers-1)]
    encoder_fin = model.get_layer(f'enc_{enc_layers-1}')
    temp = encoder_context
    for i,lay in enumerate(encoder_prev):
        temp = lay(temp)
        if dropout is not None:
            temp = model.get_layer(f'do_{i}')(temp)
     
        
    if cell == "GRU":
        enc_out, state = encoder_fin(temp)
        enc_final = [enc_out,state]

    encoder_model = keras.models.Model(encoder_inputs,enc_final)
    
    
    decoder = [model.get_layer(f'dec_{i}') for i in range(dec_layers)]
    
    attention = model.get_layer('attention')
    
    concat1 = model.get_layer('concat1')


    if cell == "GRU":
        state_inputs = []
        state_outputs = []
        
        encout_input = Input(shape=(None,nunits),name='inputenc')
        
        temp = decoder_context[:,:1,:]
                                                                  
        for i in range(dec_layers):
            state_input = Input(shape=(nunits,),name=f"inputs{i}")
            
            if i==0:
                tup = (state_input,encout_input)
                context,att_wts_out = attention(tup)
                temp = concat1([tf.expand_dims(context, 1), temp])
                
            temp,s = decoder[i](temp,initial_state = state_input)
            state_inputs.append(state_input)
            state_outputs.append(s)
            
        decoder_input_pass = [decoder_inputs,encout_input] + state_inputs

    pre_out = model.get_layer('dense1')(temp)
    final_output = model.get_layer('dense2')(pre_out)

    decoder_model = keras.models.Model(decoder_input_pass, [final_output,att_wts_out]+state_outputs)
    
    return encoder_model,decoder_model

In [ ]:
def lstm_inference(model,nunits=32,enc_layers=1,dec_layers=1,cell='LSTM',dropout=None):
    keras.backend.clear_session()
    encoder_inputs = model.input[0]
    encoder_embedding = model.get_layer('enc_embed')
    encoder_context = encoder_embedding(encoder_inputs)
    decoder_inputs = model.input[1]
    decoder_embedding = model.get_layer('dec_embed')
    decoder_context = decoder_embedding(decoder_inputs)
    
    
    encoder_prev = [model.get_layer(f'enc_{i}') for i in range(enc_layers-1)]
    encoder_fin = model.get_layer(f'enc_{enc_layers-1}')
    temp = encoder_context
    for i,lay in enumerate(encoder_prev):
        temp = lay(temp)
        if dropout is not None:
            temp = model.get_layer(f'do_{i}')(temp)
     
    if cell == "LSTM":
        enc_out, state_h,state_c = encoder_fin(temp)
        enc_final = [enc_out,state_h,state_c]
        

    encoder_model = keras.models.Model(encoder_inputs,enc_final)
    
    
    decoder = [model.get_layer(f'dec_{i}') for i in range(dec_layers)]
    
    attention = model.get_layer('attention')
    
    concat1 = model.get_layer('concat1')

    if cell == "LSTM":
        state_inputs = []
        state_outputs = []
        
        encout_input = Input(shape=(None,nunits),name='inputenc')
        
        temp = decoder_context[:,-1:,:]
                                                                  
        for i in range(dec_layers):
            decoder_input_h = Input(shape=(nunits,),name=f"inputh{i}")
            decoder_input_c = Input(shape=(nunits,),name=f"inputc{i}")
            
            if i==0:
                tup = (decoder_input_h,encout_input)
                context,att_wts_out = attention(tup)
                temp = concat1([tf.expand_dims(context, 1), temp])
                
            temp,sh,sc = decoder[i](temp,initial_state = [decoder_input_h,decoder_input_c])
            state_inputs += [decoder_input_h,decoder_input_c]
            state_outputs += [sh,sc]
            
        decoder_input_pass = [decoder_inputs,encout_input] + state_inputs

    pre_out = model.get_layer('dense1')(temp)
    final_output = model.get_layer('dense2')(pre_out)

    decoder_model = keras.models.Model(decoder_input_pass, [final_output,att_wts_out]+state_outputs)
    
    return encoder_model,decoder_model

In [ ]:
enc,dec = lstm_inference(train,nunits=256,enc_layers=3,dec_layers=1,cell="LSTM",dropout='yes')

In [ ]:
enc.save('best_enc.h5')
dec.save('best_dec.h5')

In [ ]:
def beam_search(inp,k,dec_layers,cell="LSTM"):
    enc_out = enc.predict(inp)
    statess = enc_out[1:]
    target_seq = np.zeros((inp.shape[0],1))
    target_seq[:,0] = hindi_char_map["\t"]
    if cell == "LSTM":
        states = []
        for i in range(dec_layers):
            states += [statess[0],statess[1]]
    else:
        states = []
        for i in range(dec_layers):
            states += [statess]
            
    output = dec.predict([target_seq,enc_out[0]]+states)
    states = output[2:]
    
    stat1 = np.asarray(states).transpose([1,0,2])
    
    best_chars = np.argsort(output[0][:,-1,:],axis=-1)[:,-k:]
    scores = np.sort(output[0][:,-1,:],axis=-1)[:,-k:]
    sequences = [[([ch],-np.log(sc),stat1[i],0,output[1][i]) for ch,sc in zip(best_chars[i],scores[i])] for i in range(inp.shape[0])]
    
    for t1 in range(max_hin_len-1):
        candidates = [[] for _ in range(inp.shape[0])]
        for j in range(k):
            target_seq[:,0] = [sequences[i][j][0][-1] for i in range(inp.shape[0])]
            states = list(np.asarray([sequences[i][j][2] for i in range(inp.shape[0])]).transpose([1,0,2]))
            output = dec.predict([target_seq,enc_out[0]]+states,batch_size=32)
            best_chars = np.argsort(output[0][:,-1,:],axis=-1)[:,-k:]
            scores = np.sort(output[0][:,-1,:],axis=-1)[:,-k:]
            
            stat1 = np.asarray(output[2:]).transpose([1,0,2])
            
            for i in range(inp.shape[0]):
                chk = 1 if (sequences[i][j][3]==1 or sequences[i][j][0][-1] == hindi_char_map["\n"]) else 0
                if chk == 0:
                    candidates[i] += [(sequences[i][j][0]+[best_chars[i,rep]],
                                       sequences[i][j][1]-np.log(scores[i,rep]),
                                       stat1[i],
                                       chk,
                                       np.concatenate((sequences[i][j][4],output[1][i]),axis=1))
                                      for rep in range(k)]
                else:
                    candidates[i] += [sequences[i][j]]
                    
        for i in range(inp.shape[0]):
            candidates[i] = sorted(candidates[i],key = lambda tup:tup[1]/len(tup[0]))
            sequences[i] = candidates[i][:k]
            
        print(f"decoder {t1} done")
            
        
    res = [list() for i in range(inp.shape[0])]
    att_wts = [list() for i in range(inp.shape[0])]
    for i in range(inp.shape[0]):
        for j in range(k):
            res[i].append(sequences[i][j][0])
            att_wts[i].append(sequences[i][j][4])
        
    return res,att_wts

In [ ]:
import time
start = time.time()
pred1,att_wts = beam_search(testx,5,1,cell="GRU")
print(time.time()-start)

decoder 0 done
decoder 1 done
decoder 2 done
decoder 3 done
decoder 4 done
decoder 5 done
decoder 6 done
decoder 7 done
decoder 8 done
decoder 9 done
decoder 10 done
decoder 11 done
decoder 12 done
decoder 13 done
decoder 14 done
decoder 15 done
decoder 16 done
decoder 17 done
decoder 18 done
decoder 19 done
177.1892054080963


In [ ]:
ans = []
ans1 = []
acc = 0

for i,pre in enumerate(pred1):
    word = []
    word1 = []
    
    orig = ""
    for ch in testx[i]:
        if reverse_eng_map[ch] == " ":
            break
        orig += reverse_eng_map[ch]
    word.append(orig)
    word1.append(orig)
    
    hind = ""
    for ch in testxx[i,1:]:
        if reverse_hin_map[ch] == "\n":
            break
        hind += reverse_hin_map[ch]
    
    word.append(hind)
    word1.append(hind)
    
    fl=0
    
    for j,pr in enumerate(pre):
        deco1 = ""
        for ch in pr:
            if reverse_hin_map[ch] == "\n":
                break
            deco1 += reverse_hin_map[ch]
        word.append(deco1)
        if j==0:
            word1.append(deco1)
            
        
            if hind==deco1:
                fl=1
            
    if fl==1:
        acc += 1
        
    ans.append(word)
    ans1.append(word1)
    
print(acc/len(ans))

0.02265659706796979


In [ ]:
df = pd.DataFrame(ans,columns=['English','Hindi']+[f'Hindi_pred_{i}' for i in range(5)])
df.sample(n=5)

,English,Hindi,Hindi_pred_0,Hindi_pred_1,Hindi_pred_2,Hindi_pred_3,Hindi_pred_4
1787,darmiyaan,दरमियान,दर्यामियों,दर्भानी,दर्धानी,दर्भानिय,दर्यामीय
2257,purple,पर्पल,स्रिक्र,स्रेट्ट,स्रेट्र,प्रेट्ट,स्रेक्र
2663,bandhan,बंधान,बंगल,बंधन,बंधान,बंदल,बंदला
3086,maryaadaaon,मर्यादाओं,मारायकान,माराधान,मारहाकान,मारायकां,मारायाकान
1127,char,चार,चार,चर,दीर,फाल,चाल


In [ ]:
df.to_csv('predictions_attention_beam.csv')